**Customising [LaMini-Flan-T5-783M Model] For Own Datasets**

In [ ]:
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers

**Setting the Hugging Face Hub Token**

In [ ]:
from google.colab import userdata
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] =userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


**Loading The Pdf File**

In [ ]:
# Document Loader
from PyPDF2 import PdfReader
from langchain.document_loaders import TextLoader
loader = PdfReader('./Medicine.pdf')
raw_text = ''
for i, page in enumerate(loader.pages):
          text = page.extract_text()
          if text:
              raw_text += text
documents=raw_text

In [ ]:
len(documents)

48304

In [ ]:
documents

'CHOICES FOR COMMON INFECTIONS\nnz bpac\nbetteredicin m e2013 EDITION\nRespiratory E.N.T. Eyes Skin Genito-urinary Gastrointestinal CNS\nANTIBIOTICS1\n4\n7\n8\n9\n16\n20For the updated version of this guide see:\nwww.bpac.org.nz/antibiotics\nThis edition of the guide is now out of \ndate and should no longer be used.PLEASE NOTEAntibiotic choices for common infections\nIncreasing antimicrobial resistance is now a worldwide problem, compounded by the \nlack of development of new antimicrobial medicines. This leaves the prudent use of \nantimicrobial medicines, along with infection control, as the major strategies to counter \nthis emerging threat.\nA safe and effective strategy for antibiotic use involves prescribing an antibiotic \nonly when it is needed and selecting an appropriate and effective medicine at the \nrecommended dose, with the narrowest spectrum of antimicrobial activity, fewest \nadverse effects and lowest cost.\nGeneral principles of antibiotic prescribing:\n1. Only pres

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
            separator=" ",
            chunk_size=500,
            chunk_overlap=100,
            length_function=len,
        )
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[0]

'CHOICES FOR COMMON INFECTIONS\nnz bpac\nbetteredicin m e2013 EDITION\nRespiratory E.N.T. Eyes Skin Genito-urinary Gastrointestinal CNS\nANTIBIOTICS1\n4\n7\n8\n9\n16\n20For the updated version of this guide see:\nwww.bpac.org.nz/antibiotics\nThis edition of the guide is now out of \ndate and should no longer be used.PLEASE NOTEAntibiotic choices for common infections\nIncreasing antimicrobial resistance is now a worldwide problem, compounded by the \nlack of development of new antimicrobial medicines. This'

In [ ]:
len(texts)

121

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 18.5 MB/s eta 0:00:00


**Importing The HuggingFace Embedding For Embedding Purpouse**

In [ ]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Storing the Vectors in Fassi DB**

In [ ]:
from langchain.vectorstores import FAISS

# db = FAISS.from_documents(docs, embeddings)
db = FAISS.from_texts(texts, embeddings)

**Creating Question Answer Chaining**

In [ ]:
#Create QA chain
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

In [ ]:
# Choosing a Smaller Model, for example, 'LaMini-Flan-T5-783M from HuggingFace'
llm = HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-783M", model_kwargs={"temperature": 0, "max_length": 512})

chain = load_qa_chain(llm, chain_type="stuff")
print("For Breaking the Connection Press 'd' from keyboard\n")
print("If You have Any Questions Feel free to Ask...")
while True:
  query = input("\nYou: ")
  if query=='d':
    print("Breaking the Connection ...\n")
    print("Disconnected.")
    break
  else:
    docs = db.similarity_search(query)
    print("Practo 🤓:",chain.run(input_documents=docs, question=query),end="\n")

For Breaking the Connection Press 'd' from keyboard

If You have Any Questions Feel free to Ask...

You: What is the Antibiotic for Pharyngitis?
Practo 🤓: The antibiotic for pharyngitis is Phenoxymethylpenicillin (Penicillin V).

You: And what are the Common Pathogens of Pharyngitis?
Practo 🤓: The common pathogens of Pharyngitis are Respiratory viruses, Streptococcus pneumoniae, Haemophilus influenzae, Mycoplasma pneumoniae, and bacterial infections such as pharyngitis, tonsillar swelling or exudate, and bacterial infections such as pharyngitis.

You: What is the Antibiotic of Pneumonia and for child what is the dose ?
Practo 🤓: The antibiotic of pneumonia is Amoxicillin, and for child, the dose is 25 – 30 mg/kg/dose, three times daily, for five to seven days (maximum 500 mg/dose age three months to five years, 1000 mg/dose age > five years).

You: What are the common pathogens of Otitis media?
Practo 🤓: The common pathogens of Otitis media are respiratory viruses, Streptococcus pneumo